In [1]:
import pandas as pd
import numpy as np
import sklearn as skl
from datetime import datetime
# conda install pygsheets
# pip install https://github.com/nithinmurali/pygsheets/archive/master.zip
import pygsheets
from urllib.request import urlretrieve

##import data from google sheet with spreadsheet and worksheet name
gc = pygsheets.authorize(service_file='norse-antenna-400207-696ead383a76.json')
ss = gc.open('Enterprise_Finance_Schema')
wks_input = ss.worksheet_by_title('raw_schema_generic')
data_input = wks_input.get_all_values()
headers = data_input.pop(0)
df_agreement_data = pd.DataFrame(data_input, columns=headers)
df_agreement_data['Contract_End_Date(MM/DD/YYYY)']= pd.to_datetime(df_agreement_data['Contract_End_Date(MM/DD/YYYY)'])
df_agreement_data = df_agreement_data[df_agreement_data['Contract_End_Date(MM/DD/YYYY)'] > '2025-01-01 00:00:00']


#dataset_transactions = pd.read_csv('revenue_input2.csv')
##dataset_input = pd.read_csv('enterprise_finance_input_2020_04_09.csv')
url = 'http://redash.unicommerce.com/api/queries/1703/results.csv?api_key=ZuBcefEpKuzTfKDjlzxQXDAQpMHQ34FSI4y7WRui'
urlretrieve(url, 'transactions_redash.csv') # Save file locally
df_transactions = pd.read_csv('transactions_redash.csv', sep=',')




quarter_billing_clients = pd.read_csv('http://redash.unicommerce.com/api/queries/3430/results.csv?api_key=BbvTBArTMd4s2Kna0oQBYLm944WDgQJpdwGAjH6W')
quarter_billing_clients = quarter_billing_clients['tenant_code'].tolist()
quarter_billing_data = pd.read_csv("http://redash.unicommerce.com/api/queries/3310/results.csv?api_key=gBIsYQOc3q6ZrSDv4zVconi2j5vlUJBmTSioKsny")
quarter_billing_data = quarter_billing_data[quarter_billing_data['tenant'].isin(quarter_billing_clients)]
quarter_billing_data = quarter_billing_data.drop(['quarter'],axis = 1)

#adding txn of quarterly billed clients
df_transactions = df_transactions[~df_transactions['tenant'].isin(quarter_billing_clients)]
df_transactions = pd.concat([df_transactions,quarter_billing_data], axis=0)
#df_transactions["return_expected_shipments"] = pd.to_numeric(df_transactions["return_expected_shipments"])
df_transactions[['return_expected_shipments', 'month','year','transaction_cap','soi_count','dispatched_items','invoiced_items','invoiced_shipments','dispatched_shipments','returned_shipments','reversed_shipments']] = df_transactions[['return_expected_shipments', 'month','year','transaction_cap','soi_count','dispatched_items','invoiced_items','invoiced_shipments','dispatched_shipments','returned_shipments','reversed_shipments']].astype(float)

shadow_data = pd.read_csv('http://redash.unicommerce.com/api/queries/2801/results.csv?api_key=DpvjoXlz3fJtDnwBVWyhhxonktiQzrpEnpF7x2oC')
#shadow_data = pd.read_csv('http://redash.unicommerce.com/api/queries/1716/results.csv?api_key=QwwRQkM2T5y0rGVXgJZtofaQIvUEJzN2KHZFmdj2')





df_agreement_data = df_agreement_data.replace('',np.nan)
df_agreement_data.fillna(0, inplace=True)
df_b2b_agreements = df_agreement_data[df_agreement_data.Transaction_Entity_B2B != 0][df_agreement_data.Tenant_Name != 0]
df_b2C_agreements = df_agreement_data[df_agreement_data.Transaction_Entity_B2B == 0][df_agreement_data.Tenant_Name != 0]



df_mapping = pd.DataFrame(columns = ["transactionDefinition", "dispatch","return","reversePickUp","invoice","cap"],
data = [['(invoice) (20 items)',0,0,0,1,20],
['(invoice) (1 item)',0,0,0,1,1],
['(invoice) (10 items)',0,0,0,1,10],
['(invoice) (30 items)',0,0,0,1,30],
['(invoice) (3 items)',0,0,0,1,3],
['(invoice) (5 items)',0,0,0,1,5],
['(dispatch)(1 item)',1,0,0,0,1],
['(dispatch)(20 items)',1,0,0,0,20],
['(dispatch)(10 items)',1,0,0,0,10],
['(dispatch + return )(1 item)',1,1,1,0,1],
['(dispatch + return )(2 items)',1,1,1,0,2],
['(dispatch + return )(3 items)',1,1,1,0,3],
['(dispatch + return )(4 items)',1,1,1,0,4],
['(dispatch + return )(5 items)',1,1,1,0,5],
['(dispatch + return )(6 items)',1,1,1,0,6],
['(dispatch + return )(10 items)',1,1,1,0,10],
['(dispatch + return )(15 items)',1,1,1,0,15],
['(dispatch + return )(20 items)',1,1,1,0,20],
['(dispatch + return )(50 items)',1,1,1,0,50],
['(dispatch + return )(100 items)',1,1,1,0,100],
['(invoice + return )(1 item)',0,1,1,1,1],
['(invoice + return )(2 items)',0,1,1,1,2],
['(invoice + return )(3 items)',0,1,1,1,3],
['(invoice + return )(4 items)',0,1,1,1,4],
['(invoice + return )(5 items)',0,1,1,1,5],
['(invoice + return )(6 items)',0,1,1,1,6],
['(invoice + return )(7 items)',0,1,1,1,7],
['(invoice + return )(10 items)',0,1,1,1,10],
['(invoice + return )(15 items)',0,1,1,1,15],
['(invoice + return )(20 items)',0,1,1,1,20],
['(invoice + return )(30 items)',0,1,1,1,30],
['(invoice + return )(40 items)',0,1,1,1,40],
['(invoice + return )(50 items)',0,1,1,1,50],
['(invoice + return )(80 items)',0,1,1,1,80],
['(invoice + return )(100 items)',0,1,1,1,100],
['(invoice + return )(150 items)',0,1,1,1,150],
['(invoice + cir )(5 items)',0,0,1,1,5],
['(invoice + cir )(2 items)',0,0,1,1,2],
['(invoice + cir )(1 item)',0,0,1,1,1]])

df_b2C_agreements = pd.merge(df_b2C_agreements, df_mapping, left_on = 'Transaction_Entity_B2C', right_on = 'transactionDefinition' ,how = 'left')
df_transactions_b2c = df_transactions.groupby(['tenant','transaction_cap','month','year'],as_index=False)['soi_count','dispatched_items','invoiced_items','invoiced_shipments','dispatched_shipments','returned_shipments','reversed_shipments','return_expected_shipments'].sum()


b2c_final = pd.merge(df_b2C_agreements, df_transactions_b2c, left_on = ['Tenant_Name','cap'], right_on = ['tenant','transaction_cap'] ,how = 'left')
b2c_final = b2c_final.replace('',np.nan)
b2c_final.fillna(0, inplace=True)
b2c_final['total_transaction_entity'] = b2c_final['dispatch'].astype(int)*b2c_final['dispatched_shipments'].astype(int) + b2c_final['return'].astype(int)*b2c_final['returned_shipments'].astype(int) + b2c_final['reversePickUp'].astype(int)*b2c_final['reversed_shipments'].astype(int) +  b2c_final['invoice'].astype(int)*b2c_final['invoiced_shipments'].astype(int) + b2c_final['return'].astype(int)*b2c_final['return_expected_shipments'].astype(int) 









#x_b2c = x_total_b2c[x_total_b2c.Tenant_Name != 0][x_total_b2c.B2B_Source_Channels == 0][x_total_b2c.Per_Transaction_Cap == 0]
#x_b2c['total_transaction_entity'] = x_b2c['Tag_Invoice'].astype(int)*0 + x_b2c['Tag_Dispatch_Items'].astype(int)*x_b2c['dispatched_items'].astype(int) + x_b2c['Tag_Dispatch_Shipments'].astype(int)*x_b2c['dispatched_shipment'].astype(int) +  x_b2c['Tag_Return_Items'].astype(int)*x_b2c['returned_items'].astype(int) + x_b2c['Tag_Return_Shipments'].astype(int)*x_b2c['return_shipment'].astype(int) + x_b2c['Tag_Reverse_Pickup_Items'].astype(int)*x_b2c['reverse_pickup_items'].astype(int) + x_b2c['Tag_Reverse_Pickup_Shipments'].astype(int)*x_b2c['reverse_pickup_shipment'].astype(int) + x_b2c['Tag_GRN'].astype(int)*0
#x_b2c['Contract_End_Date(MM/DD/YYYY)']= pd.to_datetime(x_b2c['Contract_End_Date(MM/DD/YYYY)'])
#x_b2c = x_b2c[x_b2c['Contract_End_Date(MM/DD/YYYY)'] > '2020-07-23 00:00:00']




#cols = ['Allowable_B2C_Transaction', 'Discount_Amount(if_any)', 'B2C_Incremental_Rate_Per_Transaction','Minimum_Monthly_Commitment','vendor_panel_count','Shadow1_Allowed','Incremental_Shadow1_Charges','Minimum_Monthly_Commitment_Shadow1','Allowable_Warehouses','warehouse_count','Additional_Charges_Per_Warehouse']
#b2c_final[cols] = b2c_final[cols].apply(pd.to_numeric, errors='coerce', axis=1)



df_tenant_b2b_channel_mapping = df_b2b_agreements.filter(['Tenant_Name','B2B_SourceCodes'], axis = 1)
df_tenant_b2b_channel_mapping['B2B_SourceCodes'].str.split(',')
df_tenant_b2b_channel_mapping = df_tenant_b2b_channel_mapping.assign(var1=df_tenant_b2b_channel_mapping['B2B_SourceCodes'].str.split(',')).explode('var1')
df_tenant_b2b_channel_mapping = df_tenant_b2b_channel_mapping.filter(['Tenant_Name','var1'], axis = 1)
df_tenant_b2b_channel_mapping = df_tenant_b2b_channel_mapping.rename(columns={"var1":"B2B_SourceCodes"})

df_b2b_agreements = pd.merge(df_b2b_agreements, df_mapping, left_on = 'Transaction_Entity_B2B', right_on = 'transactionDefinition' ,how = 'left')
df_b2b_agreements = df_b2b_agreements.rename(columns = {"transactionDefinition":"transactionDefinition_b2b","dispatch":"dispatch_b2b","return":"return_b2b","reversePickUp":"reversePickUp_b2b","invoice":"invoice_b2b","cap":"cap_b2b"})
df_b2b_agreements = pd.merge(df_b2b_agreements, df_mapping, left_on = 'Transaction_Entity_B2C', right_on = 'transactionDefinition' ,how = 'left')
df_b2b_agreements = df_b2b_agreements.rename(columns = {"transactionDefinition":"transactionDefinition_b2c","dispatch":"dispatch_b2c","return":"return_b2c","reversePickUp":"reversePickUp_b2c","invoice":"invoice_b2c","cap":"cap_b2c"})

df_b2b_transactions_b2b = pd.merge(df_tenant_b2b_channel_mapping, df_transactions, left_on = ['Tenant_Name', 'B2B_SourceCodes'], right_on = ['tenant','source_code'], how = 'left')
df_b2b_transactions_b2b = df_b2b_transactions_b2b.replace('',np.nan)
df_b2b_transactions_b2b.fillna(0, inplace=True)
df_b2b_transactions_b2b = df_b2b_transactions_b2b.groupby(['tenant','transaction_cap','month','year'], as_index=False)['soi_count','dispatched_items','invoiced_items','invoiced_shipments','dispatched_shipments','returned_shipments','reversed_shipments','return_expected_shipments'].sum()
df_b2b_transactions_b2b = df_b2b_transactions_b2b.rename(columns = {"soi_count":"soi_count_b2b","dispatched_items":"dispatched_items_b2b","invoiced_items":"invoiced_items_b2b","invoiced_shipments":"invoiced_shipments_b2b","dispatched_shipments":"dispatched_shipments_b2b","returned_shipments":"returned_shipments_b2b","reversed_shipments":"reversed_shipments_b2b","return_expected_shipments":"return_expected_shipments_b2b"})

df_tenant_b2b_channel_mapping['dumy'] = df_tenant_b2b_channel_mapping['Tenant_Name'] + df_tenant_b2b_channel_mapping['B2B_SourceCodes']
df_transactions['dumy'] = df_transactions['tenant'] + df_transactions['source_code']

df_b2c_transactions_b2b = df_transactions[~df_transactions['dumy'].isin(df_tenant_b2b_channel_mapping['dumy'])]
df_b2c_transactions_b2b = df_b2c_transactions_b2b.replace('',np.nan)
df_b2c_transactions_b2b.fillna(0, inplace=True)
df_b2c_transactions_b2b = df_b2c_transactions_b2b.groupby(['tenant','transaction_cap','month','year'])['soi_count','dispatched_items','invoiced_items','invoiced_shipments','dispatched_shipments','returned_shipments','reversed_shipments','return_expected_shipments'].sum()
df_b2c_transactions_b2b = df_b2c_transactions_b2b.rename(columns = {"soi_count":"soi_count_b2c","dispatched_items":"dispatched_items_b2c","invoiced_items":"invoiced_items_b2c","invoiced_shipments":"invoiced_shipments_b2c","dispatched_shipments":"dispatched_shipments_b2c","returned_shipments":"returned_shipments_b2c","reversed_shipments":"reversed_shipments_b2c","return_expected_shipments":"return_expected_shipments_b2c"})



b2b_final = pd.merge(df_b2b_agreements, df_b2b_transactions_b2b, left_on = ['Tenant_Name','cap_b2b'], right_on = ['tenant','transaction_cap'] ,how = 'left')
b2b_final = pd.merge(b2b_final, df_b2c_transactions_b2b, left_on = ['Tenant_Name','cap_b2c'], right_on = ['tenant','transaction_cap'] ,how = 'left')
b2b_final = b2b_final.replace('',np.nan)
b2b_final.fillna(0, inplace=True)
b2b_final['total_b2c_transaction_entity'] = b2b_final['dispatch_b2c'].astype(int)*b2b_final['dispatched_shipments_b2c'].astype(int) + b2b_final['return_b2c'].astype(int)*b2b_final['returned_shipments_b2c'].astype(int) + b2b_final['reversePickUp_b2c'].astype(int)*b2b_final['reversed_shipments_b2c'].astype(int) +  b2b_final['invoice_b2c'].astype(int)*b2b_final['invoiced_shipments_b2c'].astype(int) + b2b_final['return_b2c'].astype(int)*b2b_final['return_expected_shipments_b2c'].astype(int) 
b2b_final['total_b2b_transaction_entity'] = b2b_final['dispatch_b2b'].astype(int)*b2b_final['dispatched_shipments_b2b'].astype(int) + b2b_final['return_b2b'].astype(int)*b2b_final['returned_shipments_b2b'].astype(int) + b2b_final['reversePickUp_b2b'].astype(int)*b2b_final['reversed_shipments_b2b'].astype(int) +  b2b_final['invoice_b2b'].astype(int)*b2b_final['invoiced_shipments_b2b'].astype(int) + b2b_final['return_b2b'].astype(int)*b2b_final['return_expected_shipments_b2b'].astype(int) 

b2c_final = pd.merge(b2c_final, shadow_data, left_on = 'Tenant_Name', right_on = 'tenant', how = 'left')
b2c_final = b2c_final.replace('',np.nan)
b2c_final.fillna(0, inplace=True)
b2b_final = pd.merge(b2b_final, shadow_data, left_on = 'Tenant_Name', right_on = 'tenant', how = 'left')
b2b_final = b2b_final.replace('',np.nan)
b2b_final.fillna(0, inplace=True)



class my_dictionary(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict()  
          
    # Function to add key:value 
    def add(self, key, value): 
        self[key] = value 
        
        

def shadow_1_b2c(row, x):
    if x == 'vendor_panel':
        if b2c_final['active_vendor_panels'][row] <= b2c_final['Shadow1_Allowed'][row]:
            return b2c_final['Minimum_Monthly_Commitment_Shadow1'][row]
        elif b2c_final['active_vendor_panels'][row] > b2c_final['Shadow1_Allowed'][row]:
            return b2c_final['Minimum_Monthly_Commitment_Shadow1'][row] + (b2c_final['active_vendor_panels'][row] - b2c_final['Shadow1_Allowed'][row])*b2c_final['Incremental_Shadow1_Charges'][row]*b2c_final['Currency_Conversion_Rate'][row]
    elif x == 'dropship_panel':
        if b2c_final['active_dropship_panels'][row] <= b2c_final['Shadow1_Allowed'][row]:
            return b2c_final['Minimum_Monthly_Commitment_Shadow1'][row]
        elif b2c_final['active_dropship_panels'][row] > b2c_final['Shadow1_Allowed'][row]:
            return b2c_final['Minimum_Monthly_Commitment_Shadow1'][row] + (b2c_final['active_dropship_panels'][row] - b2c_final['Shadow1_Allowed'][row])*b2c_final['Incremental_Shadow1_Charges'][row]*b2c_final['Currency_Conversion_Rate'][row] 
    elif x == 'stores':
        if b2c_final['active_stores'][row] <= b2c_final['Shadow1_Allowed'][row]:
            return b2c_final['Minimum_Monthly_Commitment_Shadow1'][row]
        elif b2c_final['active_stores'][row] > b2c_final['Shadow1_Allowed'][row]:
            return b2c_final['Minimum_Monthly_Commitment_Shadow1'][row] + (b2c_final['active_stores'][row] - b2c_final['Shadow1_Allowed'][row])*b2c_final['Incremental_Shadow1_Charges'][row]*b2c_final['Currency_Conversion_Rate'][row] 
    elif x == 'omnichannel_stores':
        if b2c_final['active_stores'][row] <= b2c_final['Shadow1_Allowed'][row]:
            return b2c_final['Minimum_Monthly_Commitment_Shadow1'][row]
        elif b2c_final['active_stores'][row] > b2c_final['Shadow1_Allowed'][row]:
            return b2c_final['Minimum_Monthly_Commitment_Shadow1'][row] + (b2c_final['active_stores'][row] - b2c_final['Shadow1_Allowed'][row])*b2c_final['Incremental_Shadow1_Charges'][row]*b2c_final['Currency_Conversion_Rate'][row]    
    else:
        return 0
    

def shadow_1_b2b(row, x):
    if x == 'vendor_panel':
        if b2b_final['active_vendor_panels'][row] <= b2b_final['Shadow1_Allowed'][row]:
            return b2b_final['Minimum_Monthly_Commitment_Shadow1'][row]
        elif b2b_final['active_vendor_panels'][row] > b2b_final['Shadow1_Allowed'][row]:
            return b2b_final['Minimum_Monthly_Commitment_Shadow1'][row] + (b2b_final['active_vendor_panels'][row] - b2b_final['Shadow1_Allowed'][row])*b2b_final['Incremental_Shadow1_Charges'][row]*b2b_final['Currency_Conversion_Rate'][row]
    elif x == 'dropship_panel':
        if b2b_final['active_dropship_panels'][row] <= b2b_final['Shadow1_Allowed'][row]:
            return b2b_final['Minimum_Monthly_Commitment_Shadow1'][row]
        elif b2b_final['active_dropship_panels'][row] > b2b_final['Shadow1_Allowed'][row]:
            return b2b_final['Minimum_Monthly_Commitment_Shadow1'][row] + (b2b_final['active_dropship_panels'][row] - b2b_final['Shadow1_Allowed'][row])*b2b_final['Incremental_Shadow1_Charges'][row]*b2b_final['Currency_Conversion_Rate'][row] 
    elif x == 'stores':
        if b2b_final['active_stores'][row] <= b2b_final['Shadow1_Allowed'][row]:
            return b2b_final['Minimum_Monthly_Commitment_Shadow1'][row]
        elif b2b_final['active_stores'][row] > b2b_final['Shadow1_Allowed'][row]:
            return b2b_final['Minimum_Monthly_Commitment_Shadow1'][row] + (b2b_final['active_stores'][row] - b2b_final['Shadow1_Allowed'][row])*b2b_final['Incremental_Shadow1_Charges'][row]*b2b_final['Currency_Conversion_Rate'][row] 
    elif x == 'omnichannel_stores':
        if b2b_final['active_stores'][row] <= b2b_final['Shadow1_Allowed'][row]:
            return b2b_final['Minimum_Monthly_Commitment_Shadow1'][row]
        elif b2b_final['active_stores'][row] > b2b_final['Shadow1_Allowed'][row]:
            return b2b_final['Minimum_Monthly_Commitment_Shadow1'][row] + (b2b_final['active_stores'][row] - b2b_final['Shadow1_Allowed'][row])*b2b_final['Incremental_Shadow1_Charges'][row]*b2b_final['Currency_Conversion_Rate'][row]    
    else:
        return 0
 




def warehouse_revenue(value):
    if(value >= 0):
        return value
    else:
        return 0
    
    
    
def positive_number(value):
    if(value >= 0):
        return value
    else:
        return 0    
##dropship_pannel,vendor_pannel,stores,omnichannel_stores    
        
    
    
def incremental_revenue(value):
    if(value >= 0):
        return value
    else:
        return 0    
    
    
col_dtypes_b2c = b2c_final.dtypes    
cols = ['Minimum_Monthly_Commitment','Allowable_B2C_Transaction','B2C_Incremental_Rate_Per_Transaction','Minimum_Monthly_Commitment_Shadow1','Shadow1_Allowed','Incremental_Shadow1_Charges','Minimum_Monthly_Commitment_Shadow2','Shadow2_Allowed','Incremental_Shadow2_Charges','Minimum_Monthly_Commitment_Shadow3','Shadow3_Allowed','Incremental_Shadow3_Charges','Allowable_Warehouses','Additional_Charges_Per_Warehouse','Allowable_B2B_Transaction','B2B_Incremental_Rate_Per_Transaction','Staging_Server_Charges_Per_Month','Staging_Server_Count','Omnichannel_Setup_Charges',    'Hours_Spent_Physical_Training_Last_Month',    'Physical_Training_Interval_Frequency(Hours)',    'Physical_Training_Charges_Per_Interval_Frequency','Hours_Spent_Technical_Customization',    'Technical_Customization_Interval_Frequency(Hours)',    'Technical_Customization_Charges_Per_Interval_Frequency','Currency_Conversion_Rate']
b2c_final[cols] = b2c_final[cols].apply(pd.to_numeric, errors='coerce', axis=1)

col_dtypes_b2b = b2b_final.dtypes
cols = ['Minimum_Monthly_Commitment','Allowable_B2C_Transaction','B2C_Incremental_Rate_Per_Transaction','Minimum_Monthly_Commitment_Shadow1','Shadow1_Allowed','Incremental_Shadow1_Charges','Minimum_Monthly_Commitment_Shadow2','Shadow2_Allowed','Incremental_Shadow2_Charges','Minimum_Monthly_Commitment_Shadow3','Shadow3_Allowed','Incremental_Shadow3_Charges','Allowable_Warehouses','Additional_Charges_Per_Warehouse','Allowable_B2B_Transaction','B2B_Incremental_Rate_Per_Transaction','Discount_Amount(if_any)','total_b2b_transaction_entity','total_b2c_transaction_entity','Staging_Server_Charges_Per_Month','Staging_Server_Count','Omnichannel_Setup_Charges',    'Hours_Spent_Physical_Training_Last_Month',    'Physical_Training_Interval_Frequency(Hours)',    'Physical_Training_Charges_Per_Interval_Frequency','Hours_Spent_Technical_Customization',    'Technical_Customization_Interval_Frequency(Hours)',    'Technical_Customization_Charges_Per_Interval_Frequency','Currency_Conversion_Rate']    
b2b_final[cols] = b2b_final[cols].apply(pd.to_numeric, errors='coerce', axis=1)
        
        
        
month_end_revenue = my_dictionary()

for row in b2c_final.index:
    m_rev = []
    m_rev.append(b2c_final['Tenant_Name'][row]) #tenant name from agreement data
    m_rev.append(b2c_final['Transaction_Entity_B2C'][row]) #populating b2c transaction definition
    m_rev.append(b2c_final['total_transaction_entity'][row]) #populating chargable transactions into list
    m_rev.append(b2c_final['Allowable_B2C_Transaction'][row])#populating allowable transactions
    m_rev.append(incremental_revenue(b2c_final['total_transaction_entity'][row] - b2c_final['Allowable_B2C_Transaction'][row])) #incremental_transactions
    m_rev.append(b2c_final['Minimum_Monthly_Commitment'][row]*b2c_final['Currency_Conversion_Rate'][row]) #populating mmc
    m_rev.append(incremental_revenue(b2c_final['total_transaction_entity'][row] - b2c_final['Allowable_B2C_Transaction'][row]) * (b2c_final['B2C_Incremental_Rate_Per_Transaction'][row]) * (b2c_final['Currency_Conversion_Rate'][row])) #incremental_revenue
    
    m_rev.append('NA') #populating b2b transaction definition
    m_rev.append('NA') #populating b2b source codes    
    m_rev.append(0) #populating b2b chargable transactions into list
    m_rev.append(0)#populating b2b allowable transactions
    m_rev.append(0) #b2b incremental_transactions
    m_rev.append(0) #b2b incremental_revenue
            
    m_rev.append(b2c_final['Discount_Amount(if_any)'][row]) #populating discount amount in the list
    m_rev.append(b2c_final['actual_active_warehouse'][row]) #populating warehouse count
    m_rev.append(b2c_final['Allowable_Warehouses'][row]) #populating allowable warehouse 
    m_rev.append(warehouse_revenue(b2c_final['actual_active_warehouse'][row] - b2c_final['Allowable_Warehouses'][row])*b2c_final['Additional_Charges_Per_Warehouse'][row] * b2c_final['Currency_Conversion_Rate'][row]) #incremental warehouse revenue
    m_rev.append(b2c_final['Shadow1'][row]) #populating shadow 1 entity
    m_rev.append(shadow_1_b2c(row,b2c_final['Shadow1'][row])) #populating shadow 1 revenue
    m_rev.append(b2c_final['Staging_Server_Count'][row]) #populating staging server count
    m_rev.append(b2c_final['Staging_Server_Count'][row]*b2c_final['Staging_Server_Charges_Per_Month'][row] * b2c_final['Currency_Conversion_Rate'][row]) #populating staging server revenue
    m_rev.append(b2c_final['Hours_Spent_Physical_Training_Last_Month'][row]) #populating physical training hour
    m_rev.append(b2c_final['Hours_Spent_Physical_Training_Last_Month'][row]*b2c_final['Physical_Training_Charges_Per_Interval_Frequency'][row] * b2c_final['Currency_Conversion_Rate'][row]/b2c_final['Physical_Training_Interval_Frequency(Hours)'][row]) #populating physicaltraining revenue
    m_rev.append(b2c_final['Hours_Spent_Technical_Customization'][row]) #technical customization revenue
    m_rev.append(b2c_final['Hours_Spent_Technical_Customization'][row]*b2c_final['Technical_Customization_Charges_Per_Interval_Frequency'][row] * b2c_final['Currency_Conversion_Rate'][row]/b2c_final['Technical_Customization_Interval_Frequency(Hours)'][row]) #technical customization revenue revenue
    m_rev.append(int(b2c_final['MONTH'][row]))
    m_rev.append(int(b2c_final['YEAR'][row]))
    
    month_end_revenue.add(b2c_final['Tenant_Name'][row], m_rev)
    
    
for row in b2b_final.index:
    m_rev = []
    m_rev.append(b2b_final['Tenant_Name'][row]) #tenant name from agreement data
    m_rev.append(b2b_final['Transaction_Entity_B2C'][row]) #populating b2c transaction definition
    m_rev.append(b2b_final['total_b2c_transaction_entity'][row]) #populating chargable transactions into list
    m_rev.append(b2b_final['Allowable_B2C_Transaction'][row])#populating allowable transactions
    m_rev.append(incremental_revenue(b2b_final['total_b2c_transaction_entity'][row] - b2b_final['Allowable_B2C_Transaction'][row])) #incremental_transactions
    m_rev.append(b2b_final['Minimum_Monthly_Commitment'][row]) #populating mmc
    m_rev.append(incremental_revenue(b2b_final['total_b2c_transaction_entity'][row] - b2b_final['Allowable_B2C_Transaction'][row]) * b2b_final['B2C_Incremental_Rate_Per_Transaction'][row] * b2b_final['Currency_Conversion_Rate'][row]) #incremental_revenue
    
    m_rev.append(b2b_final['Transaction_Entity_B2B'][row]) #populating b2b transaction definition
    m_rev.append(b2b_final['B2B_SourceCodes'][row]) #populating b2b source codes    
    m_rev.append(b2b_final['total_b2b_transaction_entity'][row]) #populating b2b chargable transactions into list
    m_rev.append(b2b_final['Allowable_B2B_Transaction'][row])#populating b2b allowable transactions
    m_rev.append(incremental_revenue(b2b_final['total_b2b_transaction_entity'][row] - b2b_final['Allowable_B2B_Transaction'][row])) #b2b incremental_transactions
    m_rev.append(incremental_revenue(b2b_final['total_b2b_transaction_entity'][row] - b2b_final['Allowable_B2B_Transaction'][row]) * b2b_final['B2B_Incremental_Rate_Per_Transaction'][row] * b2b_final['Currency_Conversion_Rate'][row]) #b2b incremental_revenue
            
    m_rev.append(b2b_final['Discount_Amount(if_any)'][row]) #populating discount amount in the list
    m_rev.append(b2b_final['actual_active_warehouse'][row]) #populating warehouse count
    m_rev.append(b2b_final['Allowable_Warehouses'][row]) #populating allowable warehouse 
    m_rev.append(warehouse_revenue(b2b_final['actual_active_warehouse'][row] - b2b_final['Allowable_Warehouses'][row])*b2b_final['Additional_Charges_Per_Warehouse'][row] * b2b_final['Currency_Conversion_Rate'][row]) #incremental warehouse revenue
    m_rev.append(b2b_final['Shadow1'][row]) #populating shadow 1 entity
    m_rev.append(shadow_1_b2b(row,b2b_final['Shadow1'][row])) #populating shadow 1 revenue
    m_rev.append(b2b_final['Staging_Server_Count'][row]) #populating staging server count
    m_rev.append(b2b_final['Staging_Server_Count'][row]*b2b_final['Staging_Server_Charges_Per_Month'][row] * b2b_final['Currency_Conversion_Rate'][row]) #populating staging server revenue
    m_rev.append(b2b_final['Hours_Spent_Physical_Training_Last_Month'][row]) #populating physical training hour
    m_rev.append(b2b_final['Hours_Spent_Physical_Training_Last_Month'][row]*b2b_final['Physical_Training_Charges_Per_Interval_Frequency'][row] * b2b_final['Currency_Conversion_Rate'][row]/b2b_final['Physical_Training_Interval_Frequency(Hours)'][row]) #populating physicaltraining revenue
    m_rev.append(b2b_final['Hours_Spent_Technical_Customization'][row]) #technical customization revenue
    m_rev.append(b2b_final['Hours_Spent_Technical_Customization'][row]*b2b_final['Technical_Customization_Charges_Per_Interval_Frequency'][row] * b2b_final['Currency_Conversion_Rate'][row]/b2b_final['Technical_Customization_Interval_Frequency(Hours)'][row]) #technical customization revenue revenue
    m_rev.append(int(b2b_final['MONTH'][row]))
    m_rev.append(int(b2b_final['YEAR'][row]))
    
    month_end_revenue.add(b2b_final['Tenant_Name'][row], m_rev)    






df_month_end_revenue = pd.DataFrame.from_dict(month_end_revenue, 'index')        
raw_output = df_month_end_revenue.rename(columns={0: 'tenant' ,1:'b2c_transaction_def.',2: 'b2c_total_transactions' , 3:'b2c_allowable_ransactions',4: 'b2c_incremental_transactions',  5: 'MMR',6:'b2c_incremental_transaction_revenue',7:'b2b_transaction_def',8:'b2b_source_codes',9:'b2b_total_transactions',10:'b2b_allowable_ransactions',11:'b2b_incremental_transactions',12:'b2b_incremental_transaction_revenue', 13:'discount_amount', 14 : 'warehouse_count', 15:'allowable_warehouse',16:'incremental_warehouse_revenue',17:'shadow_1',18: 'shadow_1_revenue',19:'Staging_Server_Count' ,20:'Staging_Server_Revenue',21:'Hours_Spent_Physical_Training',22:'Physical_Training_Revenue',23:'Hours_Spent_Technical_Customization',24:'Technical_Customization_Revenue',25: 'month',26:'year'}) 
raw_output['updatedAt'] = str(datetime.today().strftime('%y-%m-%d:%H-%M'))
generic_raw_output = raw_output

#generic_raw_output.to_csv(r'C:\Users\Unicommerce\Documents\primarc.csv')



#======================================GENERIC B2C + B2B ENDS=======================================





def shadow_1_df_tmp(i, x):
    if x == 'vendor_panel':
        
         if df_tmp['active_vendor_panels'][i] <= df_tmp['Shadow1_Allowed'][i]:
             return df_tmp['Minimum_Monthly_Commitment_Shadow1'][i]
         elif df_tmp['active_vendor_panels'][i] > df_tmp['Shadow1_Allowed'][i]:
             return df_tmp['Minimum_Monthly_Commitment_Shadow1'][i] + (df_tmp['active_vendor_panels'][i] - df_tmp['Shadow1_Allowed'][i])*df_tmp['Incremental_Shadow1_Charges'][i]
    elif x == 'dropship_panel':
         
         if df_tmp['active_dropship_panels'][i] <= df_tmp['Shadow1_Allowed'][i]:
             return df_tmp['Minimum_Monthly_Commitment_Shadow1'][i]
         elif df_tmp['active_dropship_panels'][i] > df_tmp['Shadow1_Allowed'][i]:
             return df_tmp['Minimum_Monthly_Commitment_Shadow1'][i] + (df_tmp['active_dropship_panels'][i] - df_tmp['Shadow1_Allowed'][i])*df_tmp['Incremental_Shadow1_Charges'][i]
    
    elif x == 'stores':
        
         if df_tmp['active_stores'][i] <= df_tmp['Shadow1_Allowed'][i]:
             return df_tmp['Minimum_Monthly_Commitment_Shadow1'][i]
         elif df_tmp['active_stores'][i] > df_tmp['Shadow1_Allowed'][i]:
             return df_tmp['Minimum_Monthly_Commitment_Shadow1'][i] + (df_tmp['active_stores'][i] - df_tmp['Shadow1_Allowed'][i])*df_tmp['Incremental_Shadow1_Charges'][i]
    elif x == 'omnichannel_stores':
                  
         if df_tmp['active_stores'][i] <= df_tmp['Shadow1_Allowed'][i]:
             return df_tmp['Minimum_Monthly_Commitment_Shadow1'][i]
             return df_tmp['Minimum_Monthly_Commitment_Shadow1'][i]
         elif df_tmp['active_stores'][i] > df_tmp['Shadow1_Allowed'][i]:
             return df_tmp['Minimum_Monthly_Commitment_Shadow1'][i] + (df_tmp['active_stores'][i] - df_tmp['Shadow1_Allowed'][i])*df_tmp['Incremental_Shadow1_Charges'][i]
    else:
        return 0 







gc = pygsheets.authorize(service_file='norse-antenna-400207-696ead383a76.json')
ss = gc.open('Enterprise_Finance_Schema')
wks_input_differential = ss.worksheet_by_title('differential_demo')
data_input = wks_input_differential.get_all_values()
headers = data_input.pop(0)
df_agreement_data_differential = pd.DataFrame(data_input, columns=headers)


df_transactions2 = pd.read_csv('transactions_redash.csv', sep=',')




df_agreement_data_differential = df_agreement_data_differential.replace('',np.nan)
df_agreement_data_differential.fillna(0, inplace=True)
cols = ['Minimum_Monthly_Commitment','Allowable_B2C_Transaction_Range_Start','Allowable_B2C_Transaction_Range_End','B2C_Incremental_Rate_Per_Transaction']
df_agreement_data_differential[cols] = df_agreement_data_differential[cols].apply(pd.to_numeric, errors='coerce', axis=1)
df_b2c_agreements_differential = df_agreement_data_differential[df_agreement_data_differential.Transaction_Entity_B2B == 0][df_agreement_data_differential.Tenant_Name != 0]
df_b2b_agreements_differential = df_agreement_data_differential[df_agreement_data_differential.Transaction_Entity_B2B != 0][df_agreement_data_differential.Tenant_Name != 0]


df_b2c_agreements_differential = pd.merge(df_b2c_agreements_differential, df_mapping, left_on = 'Transaction_Entity_B2C', right_on = 'transactionDefinition' ,how = 'left')
df_transactions_differential_b2c = df_transactions2.groupby(['tenant','transaction_cap','month','year'],as_index=False)['soi_count','dispatched_items','invoiced_items','invoiced_shipments','dispatched_shipments','returned_shipments','reversed_shipments','return_expected_shipments'].sum()

b2c_differential_final = pd.merge(df_b2c_agreements_differential, df_transactions_differential_b2c, left_on = ['Tenant_Name','cap'], right_on = ['tenant','transaction_cap'] ,how = 'left')
b2c_differential_final = b2c_differential_final.replace('',np.nan)
b2c_differential_final.fillna(0, inplace=True)
b2c_differential_final['total_transaction_entity'] = b2c_differential_final['dispatch'].astype(int)*b2c_differential_final['dispatched_shipments'].astype(int) + b2c_differential_final['return'].astype(int)*b2c_differential_final['returned_shipments'].astype(int) + b2c_differential_final['reversePickUp'].astype(int)*b2c_differential_final['reversed_shipments'].astype(int) +  b2c_differential_final['invoice'].astype(int)*b2c_differential_final['invoiced_shipments'].astype(int) + b2c_differential_final['return'].astype(int)*b2c_differential_final['return_expected_shipments'].astype(int) 
b2c_differential_final['Contract_End_Date(MM/DD/YYYY)']= pd.to_datetime(b2c_differential_final['Contract_End_Date(MM/DD/YYYY)'])
b2c_differential_final = b2c_differential_final[b2c_differential_final['Contract_End_Date(MM/DD/YYYY)'] > '2025-11-01 00:00:00']

b2c_differential_final = pd.merge(b2c_differential_final, shadow_data, left_on = 'Tenant_Name', right_on = 'tenant', how = 'left')
b2c_differential_final = b2c_differential_final.replace('',np.nan)
b2c_differential_final.fillna(0, inplace=True)

cols = ['Minimum_Monthly_Commitment','Allowable_B2C_Transaction_Range_Start'    ,'Allowable_B2C_Transaction_Range_End'    ,'B2C_Incremental_Rate_Per_Transaction','Minimum_Monthly_Commitment_Shadow1','Shadow1_Allowed_Range_End','Shadow1_Allowed_Range_Start','Incremental_Shadow1_Charges','Minimum_Monthly_Commitment_Shadow2','Shadow2_Allowed','Incremental_Shadow2_Charges','Minimum_Monthly_Commitment_Shadow3','Shadow3_Allowed','Incremental_Shadow3_Charges','Allowable_Warehouses','Additional_Charges_Per_Warehouse','Allowable_B2B_Transaction','B2B_Incremental_Rate_Per_Transaction','Staging_Server_Charges_Per_Month','Staging_Server_Count','Omnichannel_Setup_Charges',    'Hours_Spent_Physical_Training_Last_Month',    'Physical_Training_Interval_Frequency(Hours)',    'Physical_Training_Charges_Per_Interval_Frequency','Hours_Spent_Technical_Customization',    'Technical_Customization_Interval_Frequency(Hours)',    'Technical_Customization_Charges_Per_Interval_Frequency','Discount_Amount(if_any)','Currency_Conversion_Rate']
b2c_differential_final[cols] = b2c_differential_final[cols].apply(pd.to_numeric, errors='coerce', axis=1)
datatype = b2c_differential_final.dtypes





class my_dictionary(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict()  
          
    # Function to add key:value 
    def add(self, key, value): 
        self[key] = value 
        
    


month_end_revenue_differential = my_dictionary()


t = b2c_differential_final.Tenant_Name.unique();
for x in t:
    print(x)
    df_tmp = b2c_differential_final[b2c_differential_final['Tenant_Name'] == x]
    df_tmp.reset_index(inplace = True, drop = True)
    temp = 0;
    m_rev = []
    for row in df_tmp.index:
    
        if ((df_tmp['total_transaction_entity'][row] >= df_tmp['Allowable_B2C_Transaction_Range_Start'][row]) & (df_tmp['total_transaction_entity'][row] <= df_tmp['Allowable_B2C_Transaction_Range_End'][row])):
            temp = temp + (df_tmp['total_transaction_entity'][row] - df_tmp['Allowable_B2C_Transaction_Range_Start'][row])*df_tmp['B2C_Incremental_Rate_Per_Transaction'][row]
            break
        else:
            temp = temp + (df_tmp['Allowable_B2C_Transaction_Range_End'][row] - df_tmp['Allowable_B2C_Transaction_Range_Start'][row])*df_tmp['B2C_Incremental_Rate_Per_Transaction'][row]

    if(df_tmp['Shadow1'][0] == 'stores'):
        col_header = 'active_stores'
    elif(df_tmp['Shadow1'][0] == 'dropship_panel'):
        col_header = 'active_dropship_panels'
    elif(df_tmp['Shadow1'][0] == 'vendor_panel'):
        col_header = 'active_vendor_panels'
    else:
        col_header = 'na'    
    
        
     
    if(col_header != 'na'):
        temp_s1 = 0;    
        for row_s1 in df_tmp.index:
            if ((df_tmp[col_header][row_s1] >= df_tmp['Shadow1_Allowed_Range_Start'][row_s1]) & (df_tmp[col_header][row_s1] <= df_tmp['Shadow1_Allowed_Range_End'][row_s1])):
                temp_s1 = temp_s1 + (df_tmp[col_header][row_s1] - df_tmp['Shadow1_Allowed_Range_Start'][row_s1])*df_tmp['Incremental_Shadow1_Charges'][row_s1]
                break
            else:
                temp_s1 = temp_s1 + (df_tmp['Shadow1_Allowed_Range_End'][row_s1] - df_tmp['Shadow1_Allowed_Range_Start'][row_s1])*df_tmp['Incremental_Shadow1_Charges'][row_s1]
    else:
        temp_s1 = 0
        
    temp_wh = 0;    
    for row_wh in df_tmp.index:
    
        if ((df_tmp['actual_active_warehouse'][row_wh] >= int(df_tmp['Allowable_Warehouses_Start'][row_wh])) & (df_tmp['actual_active_warehouse'][row_wh] <= df_tmp['Allowable_Warehouses'][row_wh])):
            temp_wh = temp_wh + (df_tmp['actual_active_warehouse'][row_wh] - int(df_tmp['Allowable_Warehouses_Start'][row_wh]))*df_tmp['Additional_Charges_Per_Warehouse'][row_wh]*df_tmp['Currency_Conversion_Rate'][row_wh]
            break
        else:
            temp_wh = temp_wh + (df_tmp['Allowable_Warehouses'][row_wh] - int(df_tmp['Allowable_Warehouses_Start'][row_wh]))*df_tmp['Additional_Charges_Per_Warehouse'][row_wh]*df_tmp['Currency_Conversion_Rate'][row_wh]

    
    
    
    m_rev.append(df_tmp['Tenant_Name'][row]) #tenant name from agreement data
    m_rev.append(df_tmp['Transaction_Entity_B2C'][row]) #populating b2c transaction definition
    m_rev.append(df_tmp['total_transaction_entity'][row]) #populating chargable transactions into list
    m_rev.append(df_tmp['Allowable_B2C_Transaction_Range_End'][0]) #populating allowable transactions
    m_rev.append(incremental_revenue(df_tmp['total_transaction_entity'][row] - df_tmp['Allowable_B2C_Transaction_Range_End'][0] + 1)) #incremental_transactions
    m_rev.append(df_tmp['Minimum_Monthly_Commitment'][0]*df_tmp['Currency_Conversion_Rate'][row]) #populating mmc
    m_rev.append(incremental_revenue(temp)*df_tmp['Currency_Conversion_Rate'][row]) #incremental_revenue
    
    m_rev.append('NA') #populating b2b transaction definition
    m_rev.append('NA') #populating b2b source codes    
    m_rev.append(0) #populating b2b chargable transactions into list
    m_rev.append(0)#populating b2b allowable transactions
    m_rev.append(0) #b2b incremental_transactions
    m_rev.append(0) #b2b incremental_revenue
            
    m_rev.append(df_tmp['Discount_Amount(if_any)'][row]*df_tmp['Currency_Conversion_Rate'][row]) #populating discount amount in the list
    m_rev.append(df_tmp['actual_active_warehouse'][row]) #populating warehouse count
    m_rev.append(df_tmp['Allowable_Warehouses_Start'][row]) #populating allowable warehouse 
    m_rev.append(temp_wh) #incremental warehouse revenue
    m_rev.append(df_tmp['Shadow1'][row]) #populating shadow 1 entity
    m_rev.append(incremental_revenue(temp_s1)) #populating shadow 1 revenue
    m_rev.append(df_tmp['Staging_Server_Count'][row]) #populating staging server count
    m_rev.append(df_tmp['Staging_Server_Count'][row]*df_tmp['Staging_Server_Charges_Per_Month'][row]*df_tmp['Currency_Conversion_Rate'][row]) #populating staging server revenue
    m_rev.append(df_tmp['Hours_Spent_Physical_Training_Last_Month'][row]) #populating physical training hour
    m_rev.append(df_tmp['Hours_Spent_Physical_Training_Last_Month'][row]*df_tmp['Currency_Conversion_Rate'][row]*df_tmp['Physical_Training_Charges_Per_Interval_Frequency'][row]/df_tmp['Physical_Training_Interval_Frequency(Hours)'][row]) #populating physicaltraining revenue
    m_rev.append(df_tmp['Hours_Spent_Technical_Customization'][row]*df_tmp['Currency_Conversion_Rate'][row]) #technical customization revenue
    m_rev.append(df_tmp['Hours_Spent_Technical_Customization'][row]*df_tmp['Currency_Conversion_Rate'][row]*df_tmp['Technical_Customization_Charges_Per_Interval_Frequency'][row]/df_tmp['Technical_Customization_Interval_Frequency(Hours)'][row]) #technical customization revenue

    m_rev.append(int(df_tmp['MONTH'][row]))
    m_rev.append(int(df_tmp['YEAR'][row]))
    month_end_revenue_differential.add(x,m_rev)
    
    
    
    
month_end_revenue_differential = pd.DataFrame.from_dict(month_end_revenue_differential, 'index')        
raw_output_differential = month_end_revenue_differential.rename(columns={0: 'tenant' ,1:'b2c_transaction_def.',2: 'b2c_total_transactions' , 3:'b2c_allowable_ransactions',4: 'b2c_incremental_transactions',  5: 'MMR',6:'b2c_incremental_transaction_revenue',7:'b2b_transaction_def',8:'b2b_source_codes',9:'b2b_total_transactions',10:'b2b_allowable_ransactions',11:'b2b_incremental_transactions',12:'b2b_incremental_transaction_revenue', 13:'discount_amount', 14 : 'warehouse_count', 15:'allowable_warehouse',16:'incremental_warehouse_revenue',17:'shadow_1',18: 'shadow_1_revenue', 19:'Staging_Server_Count' ,20:'Staging_Server_Revenue',21:'Hours_Spent_Physical_Training',22:'Physical_Training_Revenue',23:'Hours_Spent_Technical_Customization',24:'Technical_Customization_Revenue',25: 'month',26:'year'}) 
raw_output_differential['updatedAt'] = str(datetime.today().strftime('%y-%m-%d:%H-%M'))
raw_output_differential


if (raw_output_differential['b2c_incremental_transaction_revenue']['lenskart'] < raw_output_differential['MMR']['lenskart']):
    raw_output_differential['b2c_incremental_transaction_revenue']['lenskart'] = raw_output_differential['MMR']['lenskart']
    raw_output_differential['MMR']['lenskart'] = 0
    
raw_output_differential['MMR']['lenskart'] = 0    


#df_final = pd.concat([generic_raw_output,raw_output_differential], axis=0)
#df_final = df_final[df_final['month'] != 0]
#df_final.to_csv(r'C:\Users\Unicommerce\Documents\revenue_script_jun21.csv')


#raw_output_differential.to_csv(r'C:\Users\Unicommerce\Documents\raw_output_differential_v2.csv')

#======================================DIFFERENTIAL B2C & differential shadow ENDS=======================================



df_b2b_agreements_differential

df_tenant_b2b_differential_channel_mapping = df_b2b_agreements_differential.filter(['Tenant_Name','B2B_SourceCodes'], axis = 1)
df_tenant_b2b_differential_channel_mapping['B2B_SourceCodes'].str.split(',')
df_tenant_b2b_differential_channel_mapping = df_tenant_b2b_differential_channel_mapping.assign(var1=df_tenant_b2b_differential_channel_mapping['B2B_SourceCodes'].str.split(',')).explode('var1')
df_tenant_b2b_differential_channel_mapping = df_tenant_b2b_differential_channel_mapping.filter(['Tenant_Name','var1'], axis = 1)
df_tenant_b2b_differential_channel_mapping = df_tenant_b2b_differential_channel_mapping.rename(columns={"var1":"B2B_SourceCodes"})
df_tenant_b2b_differential_channel_mapping = df_tenant_b2b_differential_channel_mapping.drop_duplicates()

df_b2b_agreements_differential = pd.merge(df_b2b_agreements_differential, df_mapping, left_on = 'Transaction_Entity_B2B', right_on = 'transactionDefinition' ,how = 'left')
df_b2b_agreements_differential = df_b2b_agreements_differential.rename(columns = {"transactionDefinition":"transactionDefinition_b2b","dispatch":"dispatch_b2b","return":"return_b2b","reversePickUp":"reversePickUp_b2b","invoice":"invoice_b2b","cap":"cap_b2b"})
df_b2b_agreements_differential = pd.merge(df_b2b_agreements_differential, df_mapping, left_on = 'Transaction_Entity_B2C', right_on = 'transactionDefinition' ,how = 'left')
df_b2b_agreements_differential = df_b2b_agreements_differential.rename(columns = {"transactionDefinition":"transactionDefinition_b2c","dispatch":"dispatch_b2c","return":"return_b2c","reversePickUp":"reversePickUp_b2c","invoice":"invoice_b2c","cap":"cap_b2c"})

df_b2b_transactions_b2b_differential = pd.merge(df_tenant_b2b_differential_channel_mapping, df_transactions, left_on = ['Tenant_Name', 'B2B_SourceCodes'], right_on = ['tenant','source_code'], how = 'left')
df_b2b_transactions_b2b_differential = df_b2b_transactions_b2b_differential.replace('',np.nan)
df_b2b_transactions_b2b_differential.fillna(0, inplace=True)
df_b2b_transactions_b2b_differential = df_b2b_transactions_b2b_differential.groupby(['tenant','transaction_cap','month','year'], as_index=False)['soi_count','dispatched_items','invoiced_items','invoiced_shipments','dispatched_shipments','returned_shipments','reversed_shipments','return_expected_shipments'].sum()
df_b2b_transactions_b2b_differential = df_b2b_transactions_b2b_differential.rename(columns = {"soi_count":"soi_count_b2b","dispatched_items":"dispatched_items_b2b","invoiced_items":"invoiced_items_b2b","invoiced_shipments":"invoiced_shipments_b2b","dispatched_shipments":"dispatched_shipments_b2b","returned_shipments":"returned_shipments_b2b","reversed_shipments":"reversed_shipments_b2b","return_expected_shipments":"return_expected_shipments_b2b"})

df_tenant_b2b_differential_channel_mapping['dumy'] = df_tenant_b2b_differential_channel_mapping['Tenant_Name'] + df_tenant_b2b_differential_channel_mapping['B2B_SourceCodes']
df_transactions['dumy'] = df_transactions['tenant'] + df_transactions['source_code']

df_b2c_differential_transactions_b2b = df_transactions[~df_transactions['dumy'].isin(df_tenant_b2b_differential_channel_mapping['dumy'])]
df_b2c_differential_transactions_b2b = df_b2c_differential_transactions_b2b.replace('',np.nan)
df_b2c_differential_transactions_b2b.fillna(0, inplace=True)
df_b2c_differential_transactions_b2b = df_b2c_differential_transactions_b2b.groupby(['tenant','transaction_cap','month','year'])['soi_count','dispatched_items','invoiced_items','invoiced_shipments','dispatched_shipments','returned_shipments','reversed_shipments','return_expected_shipments'].sum()
df_b2c_differential_transactions_b2b = df_b2c_differential_transactions_b2b.rename(columns = {"soi_count":"soi_count_b2c","dispatched_items":"dispatched_items_b2c","invoiced_items":"invoiced_items_b2c","invoiced_shipments":"invoiced_shipments_b2c","dispatched_shipments":"dispatched_shipments_b2c","returned_shipments":"returned_shipments_b2c","reversed_shipments":"reversed_shipments_b2c","return_expected_shipments":"return_expected_shipments_b2c"})


df_b2b_transactions_b2b_differential['tenant'] = df_b2b_transactions_b2b_differential['tenant'].astype(str)
b2b_differential_final = pd.merge(df_b2b_agreements_differential, df_b2b_transactions_b2b_differential, left_on = ['Tenant_Name','cap_b2b'], right_on = ['tenant','transaction_cap'] ,how = 'left')
b2b_differential_final = pd.merge(b2b_differential_final, df_b2c_differential_transactions_b2b, left_on = ['Tenant_Name','cap_b2c'], right_on = ['tenant','transaction_cap'] ,how = 'left')
b2b_differential_final = b2b_differential_final.replace('',np.nan)
b2b_differential_final.fillna(0, inplace=True)
b2b_differential_final['total_b2c_transaction_entity'] = b2b_differential_final['dispatch_b2c'].astype(int)*b2b_differential_final['dispatched_shipments_b2c'].astype(int) + b2b_differential_final['return_b2c'].astype(int)*b2b_differential_final['returned_shipments_b2c'].astype(int) + b2b_differential_final['reversePickUp_b2c'].astype(int)*b2b_differential_final['reversed_shipments_b2c'].astype(int) +  b2b_differential_final['invoice_b2c'].astype(int)*b2b_differential_final['invoiced_shipments_b2c'].astype(int) + b2b_differential_final['return_b2c'].astype(int)*b2b_differential_final['return_expected_shipments_b2c'].astype(int) 
b2b_differential_final['total_b2b_transaction_entity'] = b2b_differential_final['dispatch_b2b'].astype(int)*b2b_differential_final['dispatched_shipments_b2b'].astype(int) + b2b_differential_final['return_b2b'].astype(int)*b2b_differential_final['returned_shipments_b2b'].astype(int) + b2b_differential_final['reversePickUp_b2b'].astype(int)*b2b_differential_final['reversed_shipments_b2b'].astype(int) +  b2b_differential_final['invoice_b2b'].astype(int)*b2b_differential_final['invoiced_shipments_b2b'].astype(int) + b2b_differential_final['return_b2b'].astype(int)*b2b_differential_final['return_expected_shipments_b2b'].astype(int) 
#b2b_differential_final['total_chargeable_transaction_entity'] = b2b_differential_final['total_b2c_transaction_entity'] + b2b_differential_final['total_b2b_transaction_entity'] 



b2b_differential_final = pd.merge(b2b_differential_final, shadow_data, left_on = 'Tenant_Name', right_on = 'tenant', how = 'left')
b2b_differential_final = b2b_differential_final.replace('',np.nan)
b2b_differential_final.fillna(0, inplace=True)
b2b_differential_final['Contract_End_Date(MM/DD/YYYY)']= pd.to_datetime(b2b_differential_final['Contract_End_Date(MM/DD/YYYY)'])
b2b_differential_final = b2b_differential_final[b2b_differential_final['Contract_End_Date(MM/DD/YYYY)'] > '2025-07-23 00:00:00']

cols = ['Minimum_Monthly_Commitment','total_b2c_transaction_entity','Allowable_B2C_Transaction_Range_Start','Allowable_B2C_Transaction_Range_End','B2C_Incremental_Rate_Per_Transaction','Minimum_Monthly_Commitment_Shadow1','Shadow1_Allowed_Range_Start','Shadow1_Allowed_Range_End','Incremental_Shadow1_Charges','Minimum_Monthly_Commitment_Shadow2','Shadow2_Allowed','Incremental_Shadow2_Charges','Minimum_Monthly_Commitment_Shadow3','Shadow3_Allowed','slab_no.','Incremental_Shadow3_Charges','Allowable_Warehouses','Additional_Charges_Per_Warehouse','Allowable_B2B_Transaction','B2B_Incremental_Rate_Per_Transaction','Staging_Server_Charges_Per_Month','Staging_Server_Count','Omnichannel_Setup_Charges',    'Hours_Spent_Physical_Training_Last_Month',    'Physical_Training_Interval_Frequency(Hours)',    'Physical_Training_Charges_Per_Interval_Frequency','Hours_Spent_Technical_Customization',    'Technical_Customization_Interval_Frequency(Hours)',    'Technical_Customization_Charges_Per_Interval_Frequency','Discount_Amount(if_any)']
b2b_differential_final[cols] = b2b_differential_final[cols].apply(pd.to_numeric, errors='coerce', axis=1)
datatype = b2b_differential_final.dtypes


month_end_revenue_b2b_differential = my_dictionary()


t = b2b_differential_final.Tenant_Name.unique();
for x in t:
    print(x)
    df_tmp = b2b_differential_final[b2b_differential_final['Tenant_Name'] == x]
    df_tmp.reset_index(inplace = True, drop = True)
    temp = 0;
    m_rev = []
    for row in df_tmp.index:
    
        if ((df_tmp['total_b2c_transaction_entity'][row] >= df_tmp['Allowable_B2C_Transaction_Range_Start'][row]) & (df_tmp['total_b2c_transaction_entity'][row] <= df_tmp['Allowable_B2C_Transaction_Range_End'][row])):
            temp = temp + (df_tmp['total_b2c_transaction_entity'][row] - df_tmp['Allowable_B2C_Transaction_Range_Start'][row])*df_tmp['B2C_Incremental_Rate_Per_Transaction'][row]
            break
        else:
            temp = temp + (df_tmp['Allowable_B2C_Transaction_Range_End'][row] - df_tmp['Allowable_B2C_Transaction_Range_Start'][row])*df_tmp['B2C_Incremental_Rate_Per_Transaction'][row]

    
    if(df_tmp['Shadow1'][0] == 'stores'):
        col_header = 'active_stores'
    elif(df_tmp['Shadow1'][0] == 'dropship_panel'):
        col_header = 'active_dropship_panels'
    elif(df_tmp['Shadow1'][0] == 'vendor_panel'):
        col_header = 'active_vendor_panels'
    else:
        col_header = 'na'    
    
        
     
    if(col_header != 'na'):
        temp_s1 = 0;    
        for row_s1 in df_tmp.index:
            if ((df_tmp[col_header][row_s1] >= df_tmp['Shadow1_Allowed_Range_Start'][row_s1]) & (df_tmp[col_header][row_s1] <= df_tmp['Shadow1_Allowed_Range_End'][row_s1])):
                temp_s1 = temp_s1 + (df_tmp[col_header][row_s1] - df_tmp['Shadow1_Allowed_Range_Start'][row_s1])*df_tmp['Incremental_Shadow1_Charges'][row_s1]
                break
            else:
                temp_s1 = temp_s1 + (df_tmp['Shadow1_Allowed_Range_End'][row_s1] - df_tmp['Shadow1_Allowed_Range_Start'][row_s1])*df_tmp['Incremental_Shadow1_Charges'][row_s1]
    else:
        temp_s1 = 0
    

    temp_wh = 0;    
    for row_wh in df_tmp.index:
    
        if ((df_tmp['actual_active_warehouse'][row_wh] >= int(df_tmp['Allowable_Warehouses_Start'][row_wh])) & (df_tmp['actual_active_warehouse'][row_wh] <= df_tmp['Allowable_Warehouses'][row_wh])):
            temp_wh = temp_wh + (df_tmp['actual_active_warehouse'][row_wh] - int(df_tmp['Allowable_Warehouses_Start'][row_wh]))*df_tmp['Additional_Charges_Per_Warehouse'][row_wh]*int(df_tmp['Currency_Conversion_Rate'][row_wh])
            break
        else:
            temp_wh = temp_wh + (df_tmp['Allowable_Warehouses'][row_wh] - int(df_tmp['Allowable_Warehouses_Start'][row_wh]))*df_tmp['Additional_Charges_Per_Warehouse'][row_wh]*int(df_tmp['Currency_Conversion_Rate'][row_wh])

    
    
    
    
    
    m_rev.append(df_tmp['Tenant_Name'][row]) #tenant name from agreement data
    m_rev.append(df_tmp['Transaction_Entity_B2C'][row]) #populating b2c transaction definition
    m_rev.append(df_tmp['total_b2c_transaction_entity'][row]) #populating chargable transactions into list
    m_rev.append(df_tmp['Allowable_B2C_Transaction_Range_End'][0]) #populating allowable transactions
    m_rev.append(incremental_revenue(df_tmp['total_b2c_transaction_entity'][row] - df_tmp['Allowable_B2C_Transaction_Range_End'][0])) #incremental_transactions
    m_rev.append(df_tmp['Minimum_Monthly_Commitment'][0]) #populating mmc
    m_rev.append(incremental_revenue(temp)) #incremental_revenue
    
    m_rev.append(df_tmp['Transaction_Entity_B2B'][row])#populating b2b transaction definition
    m_rev.append(df_tmp['B2B_SourceCodes'][row])#populating b2b source codes    
    m_rev.append(df_tmp['total_b2b_transaction_entity'][row]) #populating b2b chargable transactions into list
    m_rev.append(df_tmp['Allowable_B2B_Transaction'][row])#populating b2b allowable transactions
    m_rev.append(incremental_revenue(df_tmp['total_b2b_transaction_entity'][row] - df_tmp['Allowable_B2B_Transaction'][0])) #b2b incremental_transactions
    m_rev.append(incremental_revenue((df_tmp['total_b2b_transaction_entity'][row] - df_tmp['Allowable_B2B_Transaction'][0]))*df_tmp['B2B_Incremental_Rate_Per_Transaction'][row]) #b2b incremental_revenue
            
    m_rev.append(df_tmp['Discount_Amount(if_any)'][row]) #populating discount amount in the list
    m_rev.append(df_tmp['actual_active_warehouse'][row]) #populating warehouse count
    m_rev.append(df_tmp['Allowable_Warehouses_Start'][0]) #populating allowable warehouse 
    m_rev.append(temp_wh) #incremental warehouse revenue
    m_rev.append(df_tmp['Shadow1'][row]) #populating shadow 1 entity
    m_rev.append(incremental_revenue(temp_s1)) #populating shadow 1 revenue
    m_rev.append(df_tmp['Staging_Server_Count'][row]) #populating staging server count
    m_rev.append(df_tmp['Staging_Server_Count'][row]*df_tmp['Staging_Server_Charges_Per_Month'][row]) #populating staging server revenue
    m_rev.append(df_tmp['Hours_Spent_Physical_Training_Last_Month'][row]) #populating physical training hour
    m_rev.append(df_tmp['Hours_Spent_Physical_Training_Last_Month'][row]*df_tmp['Physical_Training_Charges_Per_Interval_Frequency'][row]/df_tmp['Physical_Training_Interval_Frequency(Hours)'][row]) #populating physicaltraining revenue
    m_rev.append(df_tmp['Hours_Spent_Technical_Customization'][row]) #technical customization revenue
    m_rev.append(df_tmp['Hours_Spent_Technical_Customization'][row]*df_tmp['Technical_Customization_Charges_Per_Interval_Frequency'][row]/df_tmp['Technical_Customization_Interval_Frequency(Hours)'][row]) #technical customization revenue revenue

    m_rev.append(int(df_tmp['MONTH'][row]))
    m_rev.append(int(df_tmp['YEAR'][row]))    
    
    month_end_revenue_b2b_differential.add(x,m_rev)



month_end_revenue_b2b_differential = pd.DataFrame.from_dict(month_end_revenue_b2b_differential, 'index')
month_end_revenue_b2b_differential = month_end_revenue_b2b_differential.rename(columns={0: 'tenant' ,1:'b2c_transaction_def.',2: 'b2c_total_transactions' , 3:'b2c_allowable_ransactions',4: 'b2c_incremental_transactions',  5: 'MMR',6:'b2c_incremental_transaction_revenue',7:'b2b_transaction_def',8:'b2b_source_codes',9:'b2b_total_transactions',10:'b2b_allowable_ransactions',11:'b2b_incremental_transactions',12:'b2b_incremental_transaction_revenue', 13:'discount_amount', 14 : 'warehouse_count', 15:'allowable_warehouse',16:'incremental_warehouse_revenue',17:'shadow_1',18: 'shadow_1_revenue', 19:'Staging_Server_Count' ,20:'Staging_Server_Revenue',21:'Hours_Spent_Physical_Training',22:'Physical_Training_Revenue',23:'Hours_Spent_Technical_Customization',24:'Technical_Customization_Revenue',25: 'month',26:'year'}) 
month_end_revenue_b2b_differential['updatedAt'] = str(datetime.today().strftime('%y-%m-%d:%H-%M'))
month_end_revenue_b2b_differential

#month_end_revenue_b2b_differential.to_csv(r'C:\Users\Unicommerce\Documents\month_end_revenue_b2b_differential_v2.csv')


#df_final = pd.concat([generic_raw_output,raw_output_differential,month_end_revenue_b2b_differential], axis=0)

#df_final.to_csv(r'C:\Users\Unicommerce\Documents\revenue_final.csv')


#======================================DIFFERENTIAL B2B (with differential on b2c & shadow and simple on b2b) ENDS=======================================




##import data from google sheet with spreadsheet and worksheet name
gc = pygsheets.authorize(service_file='norse-antenna-400207-696ead383a76.json')
ss = gc.open('Enterprise_Finance_Schema')
wks_input = ss.worksheet_by_title('sc_healthkart')
data_input = wks_input.get_all_values()
headers = data_input.pop(0)
df_agreement_data_special = pd.DataFrame(data_input, columns=headers)
df_agreement_data_special['Contract_End_Date(MM/DD/YYYY)']= pd.to_datetime(df_agreement_data_special['Contract_End_Date(MM/DD/YYYY)'])
df_agreement_data_special = df_agreement_data_special[df_agreement_data_special['Contract_End_Date(MM/DD/YYYY)'] > '2025-07-23 00:00:00']


df_agreement_data_special = pd.merge(df_agreement_data_special, df_mapping, left_on = 'Transaction_Entity_B2C', right_on = 'transactionDefinition' ,how = 'left')
b2c_final_healthkart = df_transactions_b2c[df_transactions_b2c['tenant'] == 'healthkart']
b2c_final_healthkart = b2c_final_healthkart.groupby(['tenant','transaction_cap','month','year'],as_index=False)['soi_count','dispatched_items','invoiced_items','invoiced_shipments','dispatched_shipments','returned_shipments','reversed_shipments','return_expected_shipments'].sum()


b2c_final_healthkart = pd.merge(df_agreement_data_special, b2c_final_healthkart, left_on = ['Tenant_Name','cap'], right_on = ['tenant','transaction_cap'] ,how = 'left')
b2c_final_healthkart = pd.merge(b2c_final_healthkart, shadow_data, left_on = 'Tenant_Name', right_on = 'tenant', how = 'left')
b2c_final_healthkart = b2c_final_healthkart.replace('',np.nan)
b2c_final_healthkart.fillna(0, inplace=True)
b2c_final_healthkart['total_transaction_entity'] = b2c_final_healthkart['dispatch'].astype(int)*b2c_final_healthkart['dispatched_shipments'].astype(int) + b2c_final_healthkart['return'].astype(int)*b2c_final_healthkart['returned_shipments'].astype(int) + b2c_final_healthkart['reversePickUp'].astype(int)*b2c_final_healthkart['reversed_shipments'].astype(int) +  b2c_final_healthkart['invoice'].astype(int)*b2c_final_healthkart['invoiced_shipments'].astype(int) + b2c_final_healthkart['return'].astype(int)*b2c_final_healthkart['return_expected_shipments'].astype(int) 

cols = ['Monthly_Estimated_Rupees','Per_Transaction_Cost_Rupees','Shadow_1_Allowed','Incremental_Shadow_1']
b2c_final_healthkart[cols] = b2c_final_healthkart[cols].apply(pd.to_numeric, errors='coerce', axis=1)

healthkart_shadow_revenue = (b2c_final_healthkart['active_dropship_panels'][0] - b2c_final_healthkart['Shadow_1_Allowed'][0])*b2c_final_healthkart['Incremental_Shadow_1'][0]


healthkart_cal1 = []
for i in range(0,len(b2c_final_healthkart.index)):
    healthkart_cal1.append(max(b2c_final_healthkart['Monthly_Estimated_Rupees'][i],b2c_final_healthkart['total_transaction_entity'][i]*b2c_final_healthkart['Per_Transaction_Cost_Rupees'][i]+healthkart_shadow_revenue))


healthkart_revenue = min(healthkart_cal1)

m_rev = []

m_rev.append('healthkart') #tenant name from agreement data
m_rev.append(b2c_final_healthkart['Transaction_Entity_B2C'][0]) #populating b2c transaction definition
m_rev.append(b2c_final_healthkart['total_transaction_entity'][0]) #populating chargable transactions into list
m_rev.append('na') #populating allowable transactions
m_rev.append('na') #incremental_transactions
m_rev.append('na') #populating mmc
m_rev.append(healthkart_revenue) #incremental_revenue
    
m_rev.append('na')#populating b2b transaction definition
m_rev.append('na')#populating b2b source codes    
m_rev.append(0) #populating b2b chargable transactions into list
m_rev.append(0)#populating b2b allowable transactions
m_rev.append(0) #b2b incremental_transactions
m_rev.append(0) #b2b incremental_revenue
            
m_rev.append(0) #populating discount amount in the list
m_rev.append(b2c_final_healthkart['actual_active_warehouse'][0]) #populating warehouse count
m_rev.append(b2c_final_healthkart['Allowable_Warehouses'][0]) #populating allowable warehouse 
m_rev.append(warehouse_revenue(b2c_final_healthkart['actual_active_warehouse'][0] - b2c_final_healthkart['Allowable_Warehouses'][0])*b2c_final_healthkart['Additional_Charges_Per_Warehouse'][0]) #incremental warehouse revenue
m_rev.append(b2c_final_healthkart['Shadow1'][0]) #populating shadow 1 entity
m_rev.append('included in incremental revenue') #populating shadow 1 revenue
m_rev.append(0) #populating staging server count
m_rev.append(0) #populating staging server revenue
m_rev.append(0) #populating physical training hour
m_rev.append(0) #populating physicaltraining revenue
m_rev.append(0) #technical customization revenue
m_rev.append(0) #technical customization revenue revenue

m_rev.append(int(b2c_final_healthkart['MONTH'][0]))
m_rev.append(int(b2c_final_healthkart['YEAR'][0]))    
    
month_end_revenue_healthkart = my_dictionary()
month_end_revenue_healthkart.add('healthkart',m_rev)
month_end_revenue_healthkart = pd.DataFrame.from_dict(month_end_revenue_healthkart, 'index')
month_end_revenue_healthkart = month_end_revenue_healthkart.rename(columns={0: 'tenant' ,1:'b2c_transaction_def.',2: 'b2c_total_transactions' , 3:'b2c_allowable_ransactions',4: 'b2c_incremental_transactions',  5: 'MMR',6:'b2c_incremental_transaction_revenue',7:'b2b_transaction_def',8:'b2b_source_codes',9:'b2b_total_transactions',10:'b2b_allowable_ransactions',11:'b2b_incremental_transactions',12:'b2b_incremental_transaction_revenue', 13:'discount_amount', 14 : 'warehouse_count', 15:'allowable_warehouse',16:'incremental_warehouse_revenue',17:'shadow_1',18: 'shadow_1_revenue', 19:'Staging_Server_Count' ,20:'Staging_Server_Revenue',21:'Hours_Spent_Physical_Training',22:'Physical_Training_Revenue',23:'Hours_Spent_Technical_Customization',24:'Technical_Customization_Revenue',25: 'month',26:'year'}) 
month_end_revenue_healthkart['updatedAt'] = str(datetime.today().strftime('%y-%m-%d:%H-%M'))
month_end_revenue_healthkart


################################################### healthkart ends#################################

# yaari_panel = pd.read_csv('http://redash.unicommerce.com/api/queries/1382/results.csv?api_key=aRqUfpK5oWV1vti3eWvbvwPiiyQapXHNXirYlEKF')
# yaari_agreement = df_agreement_data[df_agreement_data['Tenant_Name']=='yaari']
# ##active_panel_revenue   = positive_number(int((yaari_panel[yaari_panel['active_or_inactive']=='Active'][yaari_panel['tenant']=='yaari'])['seller_count']) - int(yaari_agreement['Shadow1_Allowed']))*int(yaari_agreement['Incremental_Shadow1_Charges'])
# active_panel_revenue = 0
# inactive_panel_revenue = positive_number(int((yaari_panel[yaari_panel['active_or_inactive']=='Inactive'][yaari_panel['tenant']=='yaari'])['seller_count']) - int(yaari_agreement['Shadow2_Allowed']))*int(yaari_agreement['Incremental_Shadow2_Charges'])

# generic_raw_output['shadow_1_revenue']['yaari'] = active_panel_revenue + inactive_panel_revenue

################################################### yaari panel ends#################################


df_final = pd.concat([generic_raw_output,raw_output_differential,month_end_revenue_b2b_differential,month_end_revenue_healthkart], axis=0)
df_final = df_final.sort_values(by = 'b2c_total_transactions', ascending = False)
df_final = df_final[df_final['month'] != 0]


###################################### hold account list ####################################
#installed pip via terminal before using pip here with following commands:
    #curl -O https://bootstrap.pypa.io/pip/2.7/get-pip.py
    #sudo python get-pip.py
#pip install gspread
#pip install df2gspread
#pip install gspread_dataframe
import gspread as gs
import df2gspread as d2g
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import pygsheets

gc = pygsheets.authorize(service_file='norse-antenna-400207-696ead383a76.json')
ss = gc.open('Enterprise_Finance_Schema')
wks_input = ss.worksheet_by_title('hold_clients')
data_input = wks_input.get_all_values()
headers = data_input.pop(0)
df_hold = pd.DataFrame(data_input, columns=headers)
df_hold.fillna(0, inplace=True)
df_hold = df_hold[df_hold['tenant_code']!= '']
hold_tenant_list = df_hold['tenant_code'].tolist()


################################# pending for live account list ########################

pfl_tenants = pd.read_csv('http://redash.unicommerce.com/api/queries/3219/results.csv?api_key=ybaEBiDnprxO4RwBFPaXCIXxBSfoDqR80zlciHaY')
pfl_tenant_list = pfl_tenants['tenant'].tolist()


######################################## 0 txn clients billing############################

active_tenant_list = pd.read_csv('http://redash.unicommerce.com/api/queries/1533/results.csv?api_key=WaZBdNGG4SPYpEClHXc2CdzgNQNS7CdWxGN9LzOu')
df_transactions_client_list = df_transactions_b2c[df_transactions_b2c['transaction_cap'].isin([1,20])]
active_with_txn_client =  pd.merge(active_tenant_list,df_transactions_client_list,left_on = 'code',right_on ='tenant',how = 'left')
active_with_txn_client.fillna(0, inplace=True)
active_with_0txn_client = active_with_txn_client[active_with_txn_client['transaction_cap']==0]
active_with_0txn_client["code"] = active_with_0txn_client["code"].str.lower()

df_tenant_with_mrr = df_agreement_data[["Tenant_Name","Minimum_Monthly_Commitment","Currency_Conversion_Rate"]][df_agreement_data['Contract_End_Date(MM/DD/YYYY)'] > '2025-07-23 00:00:00']
df_tenant_with_mrr["Tenant_Name"] = df_tenant_with_mrr["Tenant_Name"].str.lower()
active_with_0txn_with_agreement_client = pd.merge(active_with_0txn_client,df_tenant_with_mrr,left_on = "code",right_on="Tenant_Name",how = "left")
active_with_0txn_with_agreement_client.fillna(0, inplace=True)
active_with_0txn_with_agreement_client = active_with_0txn_with_agreement_client[active_with_0txn_with_agreement_client['Tenant_Name']!=0]
active_with_0txn_with_agreement_client = active_with_0txn_with_agreement_client[active_with_0txn_with_agreement_client['status_code']!='TEMP_TERMINATED']
active_with_0txn_with_agreement_client = active_with_0txn_with_agreement_client[["code","status_code","Minimum_Monthly_Commitment","Currency_Conversion_Rate"]]


agreement_diff = df_agreement_data_differential
agreement_diff['Contract_End_Date(MM/DD/YYYY)']= pd.to_datetime(agreement_diff['Contract_End_Date(MM/DD/YYYY)'])
agreement_diff = agreement_diff[agreement_diff['Contract_End_Date(MM/DD/YYYY)'] > '2025-07-23 00:00:00']
agreement_diff_p1 = agreement_diff.groupby(['Tenant_Name'])['Minimum_Monthly_Commitment'].agg('min')
agreement_diff_p2 = agreement_diff.groupby(['Tenant_Name'])['Currency_Conversion_Rate'].agg('min')
agreement_diff_final = pd.merge(agreement_diff_p1,agreement_diff_p2,left_on = "Tenant_Name",right_on="Tenant_Name",how = "left")


active_with_0txn_with_agreement2_client = pd.merge(active_with_0txn_client,agreement_diff_final,left_on = "code",right_on="Tenant_Name",how = "left")
active_with_0txn_with_agreement2_client.fillna(0, inplace=True)
active_with_0txn_with_agreement2_client = active_with_0txn_with_agreement2_client[active_with_0txn_with_agreement2_client["Minimum_Monthly_Commitment"]!=0]
active_with_0txn_with_agreement2_client = active_with_0txn_with_agreement2_client[active_with_0txn_with_agreement2_client['status_code']!='TEMP_TERMINATED']
active_with_0txn_with_agreement2_client = active_with_0txn_with_agreement2_client[["code","status_code","Minimum_Monthly_Commitment","Currency_Conversion_Rate"]]

final_only_mmr_billing = pd.concat([active_with_0txn_with_agreement_client,active_with_0txn_with_agreement2_client], axis=0)
final_only_mmr_billing['MMR_IN_INR'] = final_only_mmr_billing["Minimum_Monthly_Commitment"].astype(float)*final_only_mmr_billing["Currency_Conversion_Rate"].astype(str).astype(int)
final_only_mmr_billing.reset_index(inplace = True, drop = True)

active_with_0txn_client_with_fraction = active_with_0txn_client[["code","fraction"]]
final_only_pro_rata_mmr_billing = pd.merge(final_only_mmr_billing,active_with_0txn_client_with_fraction,left_on="code",right_on = "code",how = "left")
final_only_pro_rata_mmr_billing["pro_rata_MMR"] = final_only_pro_rata_mmr_billing["MMR_IN_INR"]*final_only_pro_rata_mmr_billing["fraction"]  

######## final 0 txn clients revenue ###########
final_only_pro_rata_mmr_billing_wo_hold = final_only_pro_rata_mmr_billing[~final_only_pro_rata_mmr_billing['code'].isin(list(pd.Series(hold_tenant_list).str.lower()))] 

final_only_pro_rata_mmr_billing_wo_hold.to_csv(r'C:\Users\Downloads\only_mrr_sep23.csv')

############################################# final +ve txn clients revenue #############################################
df_final_billable = df_final[~df_final['tenant'].isin(list(pd.Series(hold_tenant_list).str.lower()))]
df_final_billable.to_csv(r'C:\Users\Downloads\final_billable_sep23.csv')

############################################# revenue hold clients #####################################################
df_final_hold_with_txn = df_final[df_final['tenant'].isin(list(pd.Series(hold_tenant_list).str.lower()))]
df_final_hold_wo_txn = final_only_pro_rata_mmr_billing[final_only_pro_rata_mmr_billing['code'].isin(list(pd.Series(hold_tenant_list).str.lower()))]

###df_final_hold_with_txn.to_csv(r'C:\Users\Rakshit\Downloads\hold_with_txn_jan22.csv')
###df_final_hold_wo_txn.to_csv(r'C:\Users\Rakshit\Downloads\hold_without_txn_jan22.csv')


##################################kam input Warehouse list#########################

warehouse_db = pd.read_csv("http://redash.unicommerce.com/api/queries/2801/results.csv?api_key=DpvjoXlz3fJtDnwBVWyhhxonktiQzrpEnpF7x2oC")
warehouse_delta = pd.read_csv("http://redash.unicommerce.com/api/queries/2910/results.csv?api_key=gskZsGAznF7czZ87aSixRPJqZhB51k8UTwkrSys4")
warehouse_revenue_clients = df_final_billable[df_final_billable['incremental_warehouse_revenue'] != 0]
kam_list = pd.merge(warehouse_revenue_clients,warehouse_delta,left_on="tenant",right_on = "tenant", how = "left")
kam_account_mapping = pd.read_csv("http://redash.unicommerce.com/api/queries/1906/results.csv?api_key=p61Uvgo1a2YPi2PFz1KYwdvJA41RPQnuhmyIobu4")
kam_account_mapping['Tenant_Y'] = kam_account_mapping['Tenant_Y'].str.lower()
kam_list = pd.merge(kam_list,kam_account_mapping,left_on="tenant",right_on = "Tenant_Y", how = "left")
kam_list = pd.merge(kam_list,warehouse_db,left_on="tenant",right_on = "tenant", how = "left")
kam_list = kam_list[["AM","Manager","tenant","warehouse_count","actual_active_warehouse","wh_delta","facility_m_2","facility_m_1"]]
kam_list.fillna(0, inplace=True)
kam_list = kam_list[kam_list["wh_delta"] != 0]
kam_list = kam_list.rename(columns={'warehouse_count':'warehouse_billed','actual_active_warehouse': 'warehouse_as_per_DB','wh_delta':'WH_delta_last_2_months'}) 


# kam_list.to_csv(r'C:\Users\Rakshit\Downloads\kam_list_oct22.csv')


total_mrr = df_final_billable[df_final_billable['MMR'] != 'na']['MMR'].astype(str).astype(float)
total_mrr = total_mrr.to_frame().reset_index()
total_mrr.to_csv(r'C:\Users\Unicommerce\Documents\total_mmr.csv')

#total_mrr['MMR'].sum()

# Tennat wise MMR for Enterprise Revenue Projections

import pandas as pd
import gspread as gs
import df2gspread as d2g
from gspread_dataframe import get_as_dataframe, set_with_dataframe


gc = gs.service_account(filename='norse-antenna-400207-696ead383a76.json')
sh = gc.open_by_key('193UduGW2eNimDjep8QNNuWrrBi39Ham3T1HjCtZemrE')
worksheet = sh.get_worksheet(9)

cell_list = worksheet.range("A1:B700")
for cell in cell_list:
    cell.value = ''
worksheet.update_cells(cell_list)    

set_with_dataframe(worksheet,total_mrr)

/var/folders/hf/fghb64bn7r5_1hm8k747rzbw0000gn/T/ipykernel_7565/3537364611.py:51: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_b2b_agreements = df_agreement_data[df_agreement_data.Transaction_Entity_B2B != 0][df_agreement_data.Tenant_Name != 0]
/var/folders/hf/fghb64bn7r5_1hm8k747rzbw0000gn/T/ipykernel_7565/3537364611.py:52: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_b2C_agreements = df_agreement_data[df_agreement_data.Transaction_Entity_B2B == 0][df_agreement_data.Tenant_Name != 0]
/var/folders/hf/fghb64bn7r5_1hm8k747rzbw0000gn/T/ipykernel_7565/3537364611.py:98: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_transactions_b2c = df_transactions.groupby(['tenant','transaction_cap','month','year'],as_index=False)['soi_count','dispatched_items','invoiced_items','invoiced_shipments','dispatched_shipments','returned_shipments','reversed_

adisports
vki
abfashion
emazing
senrysa
katalysst
mcaffeine
ellana
bbpl
jalsupplychain
prozodistribution
6degree
lenskart
actiongroup
dmpl
plantex
nuawoman
teabox
vivaantafashion
puramio
napsa
donoshop
growsimplee
fablestreet
fabriqa
healthunbox
globalbees
oziva
Rapidbox
Goldfishhuc
Curefit
Stance
Goatbrands
Purvaja
Sooperrbuy
Glenmargon
Airspeed
10club
winewarehouse
Bodycupid
Monrow
Akya
unicards
Unicards
Niine
Syskaeretails
Thirumala
Bharatagri
Atsconnect
Lemoizy
ashleyandalvis
linenclub
Hydropure
Maersk
Efrontier
Khadinatural
COMICSENSE
plix
fcs
cbpl
safari
eumeworld
khadim
nudge
amalaearth
kushals
cavinkare
newfangled
dhani
neulookfashion1
wellcurve
guiltfree
cello
capl
counfreedise
ventureinnovations
banjars
scottinternational
wellversed
buildingparadise
avimeeherbal
homecandy
inkkr
bearhouse
meeshowaas
tomorrowland
jhakaashul
marketingking
ekanek
physicswallah
Dslr
duckduckbaby
putchi
fargobags
tcns
nayam
vescocycles2
nuevotech
SAPIENT HEALTH & BEAUTY PRIVATE Limited
plumgoodness

/var/folders/hf/fghb64bn7r5_1hm8k747rzbw0000gn/T/ipykernel_7565/3537364611.py:738: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  b2c_final_healthkart = b2c_final_healthkart.groupby(['tenant','transaction_cap','month','year'],as_index=False)['soi_count','dispatched_items','invoiced_items','invoiced_shipments','dispatched_shipments','returned_shipments','reversed_shipments','return_expected_shipments'].sum()
/var/folders/hf/fghb64bn7r5_1hm8k747rzbw0000gn/T/ipykernel_7565/3537364611.py:855: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_with_0txn_client["code"] = active_with_0txn_client["code"].str.lower()


In [2]:
pd.__version__

'1.5.3'